In [292]:
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [293]:
df = pd.read_csv('netflix_reviews.csv')
df = pd.concat([df['content'], df['score']], axis=1)
print('Our data set shape is:', df.shape)

Our data set shape is: (108494, 2)


In [294]:
df = df[:108000]

In [295]:
df.shape

(108000, 2)

In [296]:
df.head()

,content,score
0,I can't log in I have to pay it but I pay it s...,1
1,I love Netflix is so good I love it so much,5
2,Good,3
3,This was good when people could actually use i...,1
4,"Was working perfectly up until last month, it ...",1


In [297]:
y = df['score']
content = df['content']

### Lowering our content column

In [298]:
content = content.str.lower()

### Stopword Removal and Steamming

In [299]:
stop_words = set(stopwords.words('english'))
stop_words.add('.')
stop_words.remove('not')
pattern = r'\b[a-zA-Z]'
stemmer = PorterStemmer()
def removing_stop_words_from_content(content):
    content_list = content.split()
    filtered_content = [token for token in content_list if re.match(pattern, token)]
    removed_stop_word_content = [stemmer.stem(word) for word in filtered_content if word not in stop_words]
    return ' '.join(removed_stop_word_content)

In [300]:
content = content.apply(removing_stop_words_from_content)

In [301]:
vectorizer = CountVectorizer(max_features=1500, binary=True)
binary_matrix = vectorizer.fit_transform(content).toarray()
word_columns = vectorizer.get_feature_names_out()
word_df = pd.DataFrame(binary_matrix, columns=word_columns)

In [302]:
word_df

,aap,abil,abl,about,absolut,academia,acc,accept,access,accord,...,yearli,years,yes,yesterday,yet,you,young,youtub,youtube,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [303]:
X_train, X_test, y_train, y_test = train_test_split(word_df, y, test_size=0.3, random_state=42)

In [305]:
log_reg_model = LogisticRegression(max_iter=500)

log_reg_model.fit(X_train, y_train)

log_reg_pred = log_reg_model.predict(X_test)

accuracy_score(y_test, log_reg_pred) * 100

62.71296296296296